参考：https://zhuanlan.zhihu.com/p/27237078
# N-Gram language Modeling
首先我们介绍一下 N-Gram 模型。在一篇文章中，每一句话有很多单词组成，而对于一句话，这些单词的组成顺序也是很重要的，我们想要知道在一篇文章中我们是否可以给出几个词然后预测这些词后面的一个单词，比如’I lived in France for 10 years, I can speak _ .’那么我们想要做的就是预测最后这个词是French。

知道了我们想要做的事情之后，我们就可以引出 N-Gram 模型了。


这是一个条件概率，也就是我们给定想要预测的单词的前面几个单词，然后最大化我们想要预测的这个单词的概率。

## 数据预处理
首先我们给出了一段文章作为我们的训练集



In [1]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [3]:
len(test_sentence)

115

CONTEXT_SIZE表示我们想由前面的几个单词来预测这个单词，这里设置为2，就是说我们希望通过这个单词的前两个单词来预测这一个单词。 EMBEDDING_DIM表示word embedding的维数，上一篇已经介绍过了。



In [4]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
           for i in range(len(test_sentence)-2)]

In [6]:
trigram[:5]

[(('When', 'forty'), 'winters'),
 (('forty', 'winters'), 'shall'),
 (('winters', 'shall'), 'besiege'),
 (('shall', 'besiege'), 'thy'),
 (('besiege', 'thy'), 'brow,')]

接下来我们需要将数据整理好，也就是我们需要将单词三个分组，每个组前两个作为传入的数据，而最后一个作为预测的结果。



In [7]:
vocb = set(test_sentence) # 通过set将重复的单词去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [8]:
len(vocb)

97

In [14]:
for i,(key,value) in enumerate(word_to_idx.items()):
    print(key,value)
    if i == 5:
        break

Where 0
When 1
much 2
Proving 3
it 4
own 5


接下来需要给每个单词编码，也就是用数字来表示每个单词，这样才能够传入word embeding得到词向量。



## 定义模型


In [32]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable


class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size # 传入单词表大小
        # 通过nn.Embedding构建随机单词向量，词表大小为n_word,每个向量大小为n_dim,Embedding表为n_word*n_dim矩阵
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size*n_dim, 128) # 线性层，输入为context_size*n_dim，输出128个维度
        self.linear2 = nn.Linear(128, self.n_word) # 线性层2，输入128个维度，输出n_word个维度，预测相应的下一个词

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1) # 展平输入的一组向量，展成1维，1*(n_words*n_dim)
        out = self.linear1(emb) # 将展平向量输入线性层1
        out = F.relu(out)
        out = self.linear2(out) # 将激活后向量输入线性层2
        log_prob = F.log_softmax(out) # softmax（激活）
        return log_prob

ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
print(ngrammodel.embedding(torch.LongTensor([[1,2,4,5]])).size())
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

torch.Size([1, 4, 100])


这个模型需要传入的参数是所有的单词数，预测单词需要的前面单词数，即CONTEXT_SIZE，词向量的维度。

然后在向前传播中，首先传入单词得到词向量，比如在该模型中传入两个词，得到的词向量是(2, 100)，然后将词向量展开成(1, 200)，然后传入一个线性模型，经过relu激活函数再传入一个线性模型，输出的维数是单词总数，可以看成一个分类问题，要最大化预测单词的概率，最后经过一个log softmax激活函数。

## 训练

In [21]:
for epoch in range(100):
    print('epoch: {}'.format(epoch+1))
    print('*'*10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss.data.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))

epoch: 1
**********


C:\Users\snjl\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 4.764704
epoch: 2
**********
Loss: 4.706596
epoch: 3
**********
Loss: 4.648425
epoch: 4
**********
Loss: 4.590196
epoch: 5
**********
Loss: 4.531650
epoch: 6
**********
Loss: 4.472860
epoch: 7
**********
Loss: 4.413735
epoch: 8
**********
Loss: 4.354387
epoch: 9
**********
Loss: 4.294575
epoch: 10
**********
Loss: 4.234422
epoch: 11
**********
Loss: 4.173843
epoch: 12
**********
Loss: 4.112930
epoch: 13
**********
Loss: 4.051579
epoch: 14
**********
Loss: 3.989659
epoch: 15
**********
Loss: 3.927326
epoch: 16
**********
Loss: 3.864514
epoch: 17
**********
Loss: 3.801233
epoch: 18
**********
Loss: 3.737391
epoch: 19
**********
Loss: 3.673175
epoch: 20
**********
Loss: 3.608504
epoch: 21
**********
Loss: 3.543528
epoch: 22
**********
Loss: 3.478260
epoch: 23
**********
Loss: 3.412622
epoch: 24
**********
Loss: 3.346934
epoch: 25
**********
Loss: 3.280687
epoch: 26
**********
Loss: 3.214321
epoch: 27
**********
Loss: 3.147690
epoch: 28
**********
Loss: 3.081027
epoch: 29
**********


接着进行训练，一共跑100个epoch，在每个epoch中，word代表着预测单词的前面两个词，label表示要预测的词，然后记住需要将他们转换成Variable，接着进入网络得到结果，然后通过loss函数得到loss进行反向传播，更新参数。

可以通过预测来检测我们的模型是否有效

In [52]:
word, label = trigram[3]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)
predict_word = idx_to_word[predict_label.item()]
print('real word is {}, predict word is {}'.format(label, predict_word))

C:\Users\snjl\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


real word is thy, predict word is Thy


可以发现我们能够准确地预测这个单词。

以上我们介绍了如何通过最简单的单边 N-Gram 模型预测单词，还有一种复杂一点的N-Gram模型通过双边的单词来预测中间的单词，这种模型有个专门的名字，叫 Continuous Bag-of-Words model (CBOW)，具体的内容差别不大，就不再细讲了，代码的实现放在了github上面。